# Hermitage migration notebook

This notebook app will run the ETL process to migrate Heremitage's Odoo 11 to a new Odoo 15 system.
It will basically extract the data from v11 into csv files, make the necessary transformations to make it importable in Odoo 15 and then load the data it into Hermitage's new Odoo 15 instance.

## Initial Configurations

**Export Configurations File**
In this workspace you can find the file [export_connection.conf](export_connection.conf), it is used to set all the necessary settings and credentials to connect to Hermitage's Odoo 11 instance

**Import Configurations File**
In this workspace you can find the file [import_connection.conf](import_connection.conf), it is used to set all the necessary settings and credentials to connect to Hermitage's Odoo 15 instance

**Models Migration Configurations File**
In this workspace you can find the file [models_migration_config.conf](models_migration_config.conf). In it its possible to change certain aspects of how the models are extracted and loaded into the new system.
Most common possible configurations are:
- Fields to migrate
- Domain/Filter used to extract
- Fields to ignore when loading the data

**States Remapping File**
In this workspace you can find the file [input_csv_files/States Remapping - Input.csv](input_csv_files/States%20Remapping%20-%20Input.csv.conf), it is used to change the states and countries from certain partners.
This file is extracted from a Google Spreadsheet where some states and country data was manually corrected and then mapped with formulas to Odoo's 15 states and country base data.
Link to the Spreadsheet to check which states were affected: [States Remapping](https://docs.google.com/spreadsheets/d/1hL9S4APfv9cJYEihGn9Rtimt0xQp7mq1E0ZRpWafa8I/edit?usp=share_link)

### Imports
- pandas: to make transformations on the data
- odoo_csv_tools: Odoo SDK python package specialized for ETL processes
- Models' migration config

In [3]:
import pandas

from odoo_csv_tools import export_threaded, import_threaded
from past.translation import hooks

from models_migration_config import models_migration_config

### Constants
No need to parameterize these values for now
Included file paths for some extra csv files that don't follow the standard naming convention of model_name.csv, e.g: res.partner.csv

In [4]:
EXPORT_CONNECTION_CONFIG_DIR = 'export_connection.conf'
IMPORT_CONNECTION_CONFIG_DIR = 'import_connection.conf'
EXPORT_DEFAULT_BATCH_SIZE = 3000
IMPORT_DEFAULT_BATCH_SIZE = 1000
EXPORT_DEFAULT_REQ_CONTEXT = {}
IMPORT_DEFAULT_REQ_CONTEXT = {'tracking_disable' : True}
DEFAULT_WORKERS = 2

GENERATED_CSV_FILES_PATH = 'generated_csv_files/'
INPUT_CSV_FILES_PATH = 'input_csv_files/'
STATES_REMAPPING_FILE_NAME = 'States Remapping - Input.csv'
PARTNERS_WITHOUT_NAME_FILE_NAME =' res.partner(no name).csv'
RES_USERS_GROUPS_FILE_NAME = 'res.users(groups).csv'
CRM_TEAM_MEMBERS_FILE_NAME = 'crm.team(members).csv'

**Export function wrapper**
Created to avoid repeating code

In [1]:
def export_data(model_name = None, config = None, domain = None, fields = None, output_file = None, workers = None, batch_size = None, context = None, separator = None):
    if model_name in ['ir.model.data', 'ir.model.fields', 'res.groups']:
        model_migration_config = {}
    else:
        model_migration_config = models_migration_config[model_name]
    if not config:
        config = EXPORT_CONNECTION_CONFIG_DIR
    if not domain:
        domain = model_migration_config.get('domain', [])
    if not fields:
        fields = model_migration_config.get('fields', [])
    if output_file:
        output_file = f'{GENERATED_CSV_FILES_PATH}{output_file}'
    else:
        output_file = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
    if not workers:
        workers = DEFAULT_WORKERS
    if not batch_size:
        batch_size = model_migration_config.get('batch_size', EXPORT_DEFAULT_BATCH_SIZE)
    if not context:
        context = model_migration_config.get('context', EXPORT_DEFAULT_REQ_CONTEXT)
    if not separator:
        separator = model_migration_config.get('separator', ',')

    export_threaded.export_data(
        config,
        model_name,
        domain,
        fields,
        output=output_file,
        max_connection=workers,
        batch_size=batch_size,
        context=context,
        separator=separator,
    )

**Import function wrappers**
Created to avoid repeating code

In [5]:
def import_data(model_name = None, file_csv = None, context = None, separator = None, ignore_fields = None, group_by = None, workers = None, batch_size = None):
    model_migration_config = models_migration_config[model_name]
    if file_csv:
        file_csv = f'{GENERATED_CSV_FILES_PATH}{file_csv}'
    elif not file_csv:
        file_csv = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
    if not context:
        context = model_migration_config.get('context', IMPORT_DEFAULT_REQ_CONTEXT)
    if not separator:
        separator = model_migration_config.get('separator', ',')
    if not ignore_fields:
        ignore_fields = model_migration_config.get('ignore_fields', [])
    if not group_by:
        group_by = model_migration_config.get('group_by', False)
    if not workers:
        workers = model_migration_config.get('workers', DEFAULT_WORKERS)
    if not batch_size:
        batch_size = model_migration_config.get('batch_size', IMPORT_DEFAULT_BATCH_SIZE)

    if model_name == 'crm.lead.tag':
        model_name = 'crm.tag'

    import_threaded.import_data(
        IMPORT_CONNECTION_CONFIG_DIR,
        model_name,
        file_csv=file_csv,
        context=context,
        separator=separator,
        ignore=ignore_fields,
        split=group_by,
        max_connection=workers,
        batch_size=batch_size,
    )

def import_ignored_fields(model_name, fields = None, file_csv = None,  ignore_fields = [], group_by = [], workers = DEFAULT_WORKERS):
    model_migration_config = models_migration_config[model_name]
    if not fields:
        fields = model_migration_config['fields']
    ignore_fields = ignore_fields or model_migration_config.get('ignore_fields', [])
    if ignore_fields:
        # import the ignored fields
        added_fields = list(f for f in fields if f not in ignore_fields and f not in ['id'])
        import_data(model_name, file_csv=file_csv, ignore_fields=added_fields, group_by = group_by, workers = workers)

**Extract res.partner.category**

In [ ]:
export_data('res.partner.category')

**Load res.partner.category**

In [ ]:
import_data('res.partner.category')

**Extract and transform res.partner**

In [6]:
model_name = 'res.partner'
model_migration_config = models_migration_config[model_name]

export_data(model_name)

# Clean-up
partners_main_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
partners_dataframe = pandas.read_csv(partners_main_file_path)
partners_dataframe.rename(columns={'categ_id/id': 'category_id/id'}, inplace=True)
partners_dataframe['category_id/id'] = partners_dataframe['category_id/id'].str.replace('False', '')
partners_dataframe['type'] = partners_dataframe['type'].str.replace('False', '')
partners_dataframe['type'] = partners_dataframe['type'].str.replace('Shipping address', 'Delivery address')

# States remapping
states_remapping_file_path = f'{INPUT_CSV_FILES_PATH}{STATES_REMAPPING_FILE_NAME}'
states_remapping_dataframe = pandas.read_csv(states_remapping_file_path)

for row in states_remapping_dataframe.itertuples():
    indexes = partners_dataframe[partners_dataframe['state_id/id'] == row.flash_hlg_state_id].index.tolist()
    for i in indexes:
        state_id = row.id if type(row.id) == str else ''
        partners_dataframe.loc[i:i, 'state_id/id': 'country_id/id'] = state_id, row.country_id

partners_dataframe.to_csv(partners_main_file_path, index=False)

#Export partner without names
fields_to_export = [f for f in model_migration_config['fields'] if f != 'name']
export_data(model_name=model_name,
            domain=['|', ['name', '=', False], ['name', '=', '']],
            fields=fields_to_export,
            output_file=PARTNERS_WITHOUT_NAME_FILE_NAME
            )
partners_without_name_file_path = f'{GENERATED_CSV_FILES_PATH}{PARTNERS_WITHOUT_NAME_FILE_NAME}'
partners_dataframe = pandas.read_csv(partners_without_name_file_path)
partners_dataframe.insert(len(fields_to_export), 'name','[N/A]')
partners_dataframe.rename(columns={'categ_id/id': 'category_id/id'}, inplace=True)
partners_dataframe['type'] = partners_dataframe['type'].str.replace('Shipping address', 'Delivery address')
partners_dataframe.to_csv(partners_without_name_file_path, index=False)

time for batch 0: 11.171121597290039
time for batch 1: 11.353484869003296
time for batch 2: 11.011966466903687
time for batch 3: 10.838736295700073
time for batch 11: 11.1662917137146
time for batch 4: 11.343334913253784
time for batch 12: 10.514366149902344
time for batch 13: 10.526703596115112
time for batch 8: 10.554396152496338
time for batch 9: 10.537352323532104
time for batch 10: 10.353218793869019
time for batch 5: 10.691975116729736
time for batch 6: 10.155412912368774
time for batch 7: 10.135194301605225
time for batch 14: 5.3277428150177
43316 res.partner exported, total time 82.4939239025116 second(s)
Writing file
time for batch 0: 1.950087070465088
290 res.partner exported, total time 2.6312451362609863 second(s)
Writing file


**Load res.partner**

In [ ]:
model_name = 'res.partner'
#import partners without name
import_data(model_name=model_name, file_csv=PARTNERS_WITHOUT_NAME_FILE_NAME)
#import the rest of the partners
import_data(model_name=model_name, group_by='parent_id/id', workers=1)

ignore_fields = ['id', 'name'] + models_migration_config['res.partner']['ignore_fields']
import_ignored_fields(model_name, file_csv=PARTNERS_WITHOUT_NAME_FILE_NAME, ignore_fields = ignore_fields, workers=1)

**Extract and transform res.users**

In [ ]:
model_name = 'res.users'
export_data(model_name)

res_users_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
users_dataframe = pandas.read_csv(res_users_file_path)
users_dataframe.rename(columns={'image': 'image_1920'}, inplace=True)
users_dataframe.to_csv(res_users_file_path, index=False)

# Export users security groups
fields_to_export = ['id', 'groups_id', 'groups_id/id']
export_data(model_name=model_name,
            fields=fields_to_export,
            output_file=RES_USERS_GROUPS_FILE_NAME,
            )
res_users_groups_file_path = f'{GENERATED_CSV_FILES_PATH}{RES_USERS_GROUPS_FILE_NAME}'
users_groups_dataframe = pandas.read_csv(res_users_groups_file_path)
users_groups_dataframe.fillna(method='ffill', inplace=True)

# Export v15 groups
export_data(config='import_connection.conf',
            model_name='res.groups',
            fields=['id'],
            output_file='res.groups.csv',
            )
res_groups_file_path = f'{GENERATED_CSV_FILES_PATH}res.groups.csv'
groups_dataframe = pandas.read_csv(res_groups_file_path)
groups_dataframe.rename(columns={'id': 'groups_id/id'}, inplace=True)
users_groups_dataframe = users_groups_dataframe.merge(groups_dataframe, on='groups_id/id', how='inner')

users_groups_dataframe.to_csv(res_users_groups_file_path, index=False)

**Load res.users** (and load remaining res.partner fields)

In [7]:
model_name = 'res.users'
import_data(model_name=model_name)
import_data(model_name=model_name, ignore_fields=['groups_id'], file_csv=RES_USERS_GROUPS_FILE_NAME, group_by='id', workers=1, context={'update_many2many': True})

#Import remaining res.partner fields
ignore_fields = ['id', 'name'] + models_migration_config['res.partner']['ignore_fields']
import_ignored_fields('res.partner', ignore_fields = ignore_fields, workers=1)

open generated_csv_files/res.partner.csv


open generated_csv_files/res.partner.csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 1000 : 30.44633436203003
time for batch [1] - 1000 of 1000 : 15.47355341911316
time for batch [2] - 1000 of 1000 : 11.03710389137268
time for batch [3] - 1000 of 1000 : 11.27837586402893
time for batch [4] - 1000 of 1000 : 10.11946988105774
time for batch [5] - 1000 of 1000 : 9.634913682937622
time for batch [6] - 1000 of 1000 : 10.613754510879517
time for batch [10] - 1000 of 1000 : 9.936044692993164
time for batch [8] - 1000 of 1000 : 10.727485656738281
time for batch [12] - 1000 of 1000 : 9.929571390151978
time for batch [7] - 1000 of 1000 : 9.099654197692871
time for batch [11] - 1000 of 1000 : 10.261418104171753
time for batch [9] - 1000 of 1000 : 10.728813409805298
time for batch [13] - 1000 of 1000 : 9.810096740722656
time for batch [14] - 1000 of 1000 : 9.935624837875366
time for batch [18] - 1000 of 1000 : 10.277380228042603
time for batch [16] - 1000 of 1000 : 10.070571899414062
time for batch [17] - 1000 of 1000 : 1

**Extract crm.lead.tag**

In [60]:
export_data('crm.lead.tag')

time for batch 0: 0.7539858818054199
27 crm.lead.tag exported, total time 1.936544418334961 second(s)
Writing file


**Load crm.lead.tag**

In [61]:
import_data('crm.lead.tag')

open generated_csv_files/crm.lead.tag.csv


open generated_csv_files/crm.lead.tag.csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 27 : 0.06116127967834473
27 crm.tag imported, total time 0.06213235855102539 second(s)


**Extract crm.stage**

In [62]:
export_data('crm.stage')

time for batch 0: 0.7674407958984375
20 crm.stage exported, total time 1.5480220317840576 second(s)
Writing file


**Load crm.stage**

In [63]:
import_data('crm.stage')

open generated_csv_files/crm.stage.csv


open generated_csv_files/crm.stage.csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 20 : 0.057265520095825195
20 crm.stage imported, total time 0.05883026123046875 second(s)


**Extract and transform crm.team**

In [64]:
model_name = 'crm.team'
export_data(model_name)

#Export team members
fields_to_export = ['id', 'member_ids', 'member_ids/id']
export_data(model_name=model_name,
            fields=fields_to_export,
            output_file=CRM_TEAM_MEMBERS_FILE_NAME
            )
crm_team_file_path = f'{GENERATED_CSV_FILES_PATH}{CRM_TEAM_MEMBERS_FILE_NAME}'
crm_team_dataframe = pandas.read_csv(crm_team_file_path)
crm_team_dataframe.fillna(method='ffill', inplace=True)
crm_team_dataframe.to_csv(crm_team_file_path, index=False)

time for batch 0: 1.0742380619049072
129 crm.team exported, total time 1.8563196659088135 second(s)
Writing file
time for batch 0: 1.6371424198150635
129 crm.team exported, total time 2.3808932304382324 second(s)
Writing file


**Load crm.team**

In [65]:
model_name = 'crm.team'
import_data(model_name)
import_data(model_name=model_name, ignore_fields=['member_ids'], file_csv=CRM_TEAM_MEMBERS_FILE_NAME, workers=1, context={'update_many2many': True})

open generated_csv_files/crm.team.csv


open generated_csv_files/crm.team.csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 129 : 0.4250047206878662
129 crm.team imported, total time 0.4266352653503418 second(s)
open generated_csv_files/crm.team(members).csv


open generated_csv_files/crm.team(members).csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 1000 : 9.994344472885132
time for batch [1] - 1000 of 583 : 5.6810994148254395
1583 crm.team imported, total time 15.6790030002594 second(s)


**Extract and transform crm.lead**

In [66]:
model_name = 'crm.lead'
export_data(model_name)

crm_lead_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
crm_lead_dataframe = pandas.read_csv(crm_lead_file_path)
crm_lead_dataframe.rename(columns={'planned_revenue': 'expected_revenue'}, inplace=True)
crm_lead_dataframe['priority'] = crm_lead_dataframe['priority'].str.replace('Low', 'Medium')
crm_lead_dataframe['priority'] = crm_lead_dataframe['priority'].str.replace('Normal', 'Low')
crm_lead_dataframe['tag_ids/id'] = crm_lead_dataframe['tag_ids/id'].str.replace('False', '')
crm_lead_dataframe.sort_values('id', inplace=True)

crm_lead_dataframe.to_csv(crm_lead_file_path, index=False)

time for batch 0: 13.550244092941284
time for batch 1: 14.136132001876831
time for batch 3: 13.071030616760254
time for batch 2: 13.708263635635376
time for batch 4: 13.520424127578735
14908 crm.lead exported, total time 42.08841586112976 second(s)
Writing file


**Load crm.lead**

In [67]:
import_data('crm.lead')

open generated_csv_files/crm.lead.csv


open generated_csv_files/crm.lead.csv


Skipping until line 0 excluded
time for batch [0] - [__export__.crm_stage_13_0a60bb34] - 1000 of 1604 : 4.314177989959717
time for batch [0] - [__export__.crm_stage_13_0a60bb34] - 2000 of 1604 : 2.5542542934417725
time for batch [1] - [__export__.crm_stage_56_6433748d] - 1000 of 1009 : 4.292686939239502
time for batch [1] - [__export__.crm_stage_56_6433748d] - 2000 of 1009 : 0.08962035179138184
time for batch [2] - [crm.stage_lead1] - 1000 of 9872 : 4.2356040477752686
time for batch [2] - [crm.stage_lead1] - 2000 of 9872 : 4.211527585983276
time for batch [2] - [crm.stage_lead1] - 3000 of 9872 : 3.941403388977051
time for batch [2] - [crm.stage_lead1] - 4000 of 9872 : 4.325936794281006
time for batch [2] - [crm.stage_lead1] - 5000 of 9872 : 4.129857063293457
time for batch [2] - [crm.stage_lead1] - 6000 of 9872 : 4.2307679653167725
time for batch [2] - [crm.stage_lead1] - 7000 of 9872 : 4.260603666305542
time for batch [2] - [crm.stage_lead1] - 8000 of 9872 : 4.255374908447266
time for

**Extract and transform mail.message**
Requires all the previous steps to work properly given that it depends on the new external ids created on the new Odoo instance

In [68]:
model_name = 'mail.message'
export_data(model_name)

mail_message_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
mail_message_dataframe = pandas.read_csv(mail_message_file_path)

mail_message_dataframe['subtype_id/id'] = mail_message_dataframe['subtype_id/id'].str.replace('False', '')
mail_message_dataframe['partner_ids/id'] = mail_message_dataframe['partner_ids/id'].str.replace('False', '')

models_with_mail_messages = list(mail_message_dataframe['model'].unique())
models_to_export_external_ids = [m for m in models_with_mail_messages if m in models_migration_config.keys()]
#Export external_ids
fields_to_export = ['complete_name', 'res_id', 'model']
export_data(model_name='ir.model.data',
            fields=fields_to_export,
            output_file='ir.model.data.old.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
#Export v15 external_ids
export_data(config='import_connection.conf',
            model_name='ir.model.data',
            fields=fields_to_export,
            output_file='ir.model.data.new.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
external_ids_old_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.old.csv'
external_ids_new_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.new.csv'
merged_data_frame_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.merged.csv'
ir_model_data_old_dataframe = pandas.read_csv(external_ids_old_file_path)
ir_model_data_new_dataframe = pandas.read_csv(external_ids_new_file_path)
ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.partner_root', 'base.partner_admin')
ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.default_user_res_partner', 'base.template_portal_user_id_res_partner')
ir_model_data_old_dataframe.to_csv(external_ids_old_file_path, index=False)
# Merge old and new external ids to extract the new database id
ir_model_data_old_dataframe.rename(columns={'res_id': 'old_res_id'}, inplace=True)
ir_model_data_merged_dataframe = ir_model_data_new_dataframe.merge(ir_model_data_old_dataframe, on=['complete_name', 'model'], how='inner')
ir_model_data_merged_dataframe.rename(columns={'res_id': 'new_res_id'}, inplace=True)
ir_model_data_merged_dataframe['old_res_id'] = ir_model_data_merged_dataframe['old_res_id'].astype('int')
ir_model_data_merged_dataframe.to_csv(merged_data_frame_file_path, index=False)

# Merge the mail.message dataframe with ir_model_data_merged_dataframe to get the new database ids
mail_message_dataframe = mail_message_dataframe.merge(ir_model_data_merged_dataframe, left_on=['model', 'res_id'], right_on=['model', 'old_res_id'], how='inner')
mail_message_dataframe.rename(columns={'res_id': 'old_res_id'}, inplace=True)
mail_message_dataframe.rename(columns={'new_res_id': 'res_id'}, inplace=True)
mail_message_dataframe['res_id'] = mail_message_dataframe['res_id'].astype('int')

mail_message_dataframe.sort_values('date', ascending=True, inplace=True)
mail_message_dataframe.to_csv(mail_message_file_path, index=False)

time for batch 1: 8.822322845458984
time for batch 0: 9.015714645385742
time for batch 2: 7.847066640853882
time for batch 3: 8.698562145233154
time for batch 4: 8.105543851852417
time for batch 5: 8.09717082977295
time for batch 6: 7.8463099002838135
time for batch 7: 7.452922344207764
time for batch 8: 8.477424383163452
time for batch 9: 7.878674745559692
time for batch 10: 8.659231662750244
time for batch 17: 8.943337440490723
time for batch 13: 7.866714954376221
time for batch 18: 8.27489185333252
time for batch 14: 9.17553162574768
time for batch 15: 9.453514814376831
time for batch 11: 8.22123384475708
time for batch 22: 8.6731698513031
time for batch 19: 8.417027711868286
time for batch 12: 9.047569274902344
time for batch 21: 7.726343870162964
time for batch 20: 8.355023860931396
time for batch 16: 7.95101261138916
time for batch 29: 8.535937309265137
time for batch 30: 7.89205002784729
time for batch 31: 7.852519989013672
time for batch 33: 7.330997467041016
time for batch 26:

/tmp/ipykernel_40468/842229724.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.partner_root', 'base.partner_admin')
/tmp/ipykernel_40468/842229724.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.default_user_res_partner', 'base.template_portal_user_id_res_partner')


**Load mail.message**

In [69]:
model_name = 'mail.message'
ignored_fields = ['old_res_id', 'old_res_id2', 'complete_name', 'parent_id/id']
import_data(model_name=model_name, ignore_fields=ignored_fields, group_by='complete_name')

open generated_csv_files/mail.message.csv


open generated_csv_files/mail.message.csv


Skipping until line 0 excluded
time for batch [1] - [__export__.crm_lead_20872_30733915] - 1000 of 1007 : 2.9201033115386963
time for batch [0] - [__export__.crm_lead_19460_de380ae3] - 1000 of 1000 : 3.0580949783325195
time for batch [1] - [__export__.crm_lead_20872_30733915] - 2000 of 1007 : 0.137101411819458
time for batch [2] - [__export__.crm_lead_22248_fcbdf636] - 1000 of 1413 : 3.2462687492370605
time for batch [3] - [__export__.crm_lead_22252_5debacbb] - 1000 of 3228 : 3.390671491622925
time for batch [2] - [__export__.crm_lead_22248_fcbdf636] - 2000 of 1413 : 1.5159623622894287
time for batch [3] - [__export__.crm_lead_22252_5debacbb] - 2000 of 3228 : 2.8260722160339355
time for batch [4] - [__export__.crm_lead_22259_3fd22dac] - 1000 of 1135 : 2.863431215286255
time for batch [4] - [__export__.crm_lead_22259_3fd22dac] - 2000 of 1135 : 0.37528014183044434
time for batch [3] - [__export__.crm_lead_22252_5debacbb] - 3000 of 3228 : 3.317457437515259
time for batch [3] - [__export__

**Extract and transform mail.tracking.value**

In [ ]:
model_name = 'mail.tracking.value'
mail_tracking_value_all_file_name = f'{model_name}.all.csv'
mail_tracking_value_all_file_path = f'{GENERATED_CSV_FILES_PATH}{mail_tracking_value_all_file_name}'
mail_tracking_value_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
export_data(model_name=model_name, output_file=mail_tracking_value_all_file_name)
mail_tracking_value_dataframe = pandas.read_csv(mail_tracking_value_all_file_path, low_memory=False)

mail_message_file_path = f'{GENERATED_CSV_FILES_PATH}mail.message.csv'
mail_message_dataframe = pandas.read_csv(mail_message_file_path, low_memory=False)
mail_message_dataframe.rename(columns={'id': 'mail_message_id/id'}, inplace=True)

#Merge tracking values with mail.message to filter the tracking values to import and adding the model column
mail_message_dataframe = mail_message_dataframe[['model', 'mail_message_id/id']]
mail_tracking_value_dataframe = mail_tracking_value_dataframe.merge(mail_message_dataframe, on='mail_message_id/id', how='inner')

#Export v15 ir_model_fields
fields_model_name = 'ir.model.fields'
models_with_mail_messages = list(mail_message_dataframe['model'].unique())
models_to_export_external_ids = [m for m in models_with_mail_messages if m in models_migration_config.keys()]
export_data(config='import_connection.conf',
            model_name=fields_model_name,
            fields=['id', 'name', 'model'],
            output_file=f'{fields_model_name}.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
ir_model_fields_file_path = f'{GENERATED_CSV_FILES_PATH}{fields_model_name}.csv'
fields_dataframe = pandas.read_csv(ir_model_fields_file_path)
fields_dataframe.rename(columns={'id': 'field/id'}, inplace=True)
fields_dataframe.rename(columns={'name': 'field'}, inplace=True)

#Merge tracking values with fields to extract the fields' external ids
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('planned_revenue', 'expected_revenue')
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('categ_id', 'category_id')
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('salesperson_ids', 'user_id')
mail_tracking_value_dataframe = mail_tracking_value_dataframe.merge(fields_dataframe, on=['field', 'model'], how='inner')
# Drop columns used just to merge
mail_tracking_value_dataframe.drop(columns={'field', 'model'}, inplace=True)

mail_tracking_value_dataframe.to_csv(mail_tracking_value_file_path, index=False)

time for batch 0: 5.2004406452178955
time for batch 1: 5.222038269042969
time for batch 2: 5.125707626342773
time for batch 3: 5.096868991851807
time for batch 4: 4.798511028289795
time for batch 5: 4.973243951797485
time for batch 6: 4.84392237663269
time for batch 7: 4.6991119384765625
time for batch 9: 6.069331884384155
time for batch 14: 6.312368869781494
time for batch 10: 4.905071258544922
time for batch 17: 4.868776559829712
time for batch 13: 4.812799692153931
time for batch 18: 4.987366437911987
time for batch 8: 4.652878999710083
time for batch 15: 4.9380857944488525
time for batch 16: 4.599104166030884
time for batch 11: 4.572903871536255
time for batch 12: 4.632498264312744
time for batch 19: 4.626989841461182
time for batch 20: 4.77591609954834
time for batch 21: 4.684791088104248
time for batch 22: 4.850350618362427
time for batch 23: 4.722278118133545
time for batch 24: 5.0333170890808105
time for batch 25: 5.034696340560913
time for batch 26: 4.5005199909210205
time for

**Load mail.tracking.value**

In [ ]:
import_data('mail.tracking.value')